In [1]:
# Install the openpyxl library
# openpyxl is a library that allows you to read and write Excel files (xlsx format) with pandas.
!pip install openpyxl
!pip install -q faker


In [3]:
# Import the required libraries
import pandas as pd        # pandas is used for data manipulation and analysis
import hashlib             # hashlib is used for hashing values (e.g., for anonymization)
import re                  # re is the Python library for working with regular expressions
import random
from faker import Faker
from datetime import datetime  # datetime is used for handling date and time operations
import ast                 # ast is used for safely evaluating strings containing Python expressions (like lists)
from datetime import timedelta


# Define the path to the Excel file
path = "Data/mobile_customers.xlsx"

# Read the Excel file into a pandas DataFrame
# openpyxl engine (installed with 'pip install openpyxl') is automatically used to read .xlsx files
df = pd.read_excel(path)

# Display the first 5 rows of the DataFrame to check the data
df.head()

,Unnamed: 0,customer_id,date_registered,username,name,gender,address,email,birthdate,current_location,residence,employer,job,age,salary,credit_card_provider,credit_card_number,credit_card_security_code,credit_card_expire
0,0,24c9d2d0-d0d3-4a90-9a3a-e00e4aac99bd,2021-09-29,robertsbryan,Jonathan Snyder,M,"24675 Susan Valley\nNorth Dianabury, MO 02475",marcus58@hotmail.com,1978-03-11,"['78.937112', '71.260464']","195 Brandi Junctions\nNew Julieberg, NE 63410","Byrd, Welch and Holt",Chief Technology Officer,49,53979,VISA 19 digit,38985874269846,994,2023-10-27 00:00:00
1,1,7b2bc220-0296-4914-ba46-d6cc6a55a62a,2019-08-17,egarcia,Susan Dominguez,F,"4212 Cheryl Inlet\nPort Davidmouth, NC 54884",alexanderkathy@hotmail.com,1970-11-29,"['-24.1692185', '100.746122']","58272 Brown Isle Apt. 698\nPort Michael, HI 04693",Hurst PLC,Data scientist,43,81510,Discover,6525743622515979,163,2023-07-30 00:00:00
2,2,06febdf9-07fb-4a1b-87d7-a5f97d9a5faf,2019-11-01,turnermegan,Corey Hebert,M,"07388 Coleman Prairie\nLake Amy, IA 78695",vwood@gmail.com,2009-04-23,"['8.019908', '-19.603269']","36848 Jones Lane Suite 282\nMarquezbury, ID 26822","Mora, Caldwell and Guerrero",Chief Operating Officer,47,205345,VISA 16 digit,4010729915028682247,634,2023-04-26 00:00:00
3,3,23df88e5-5dd3-46af-ac0d-0c6bd92e4b96,2021-12-31,richardcampbell,Latasha Griffin,F,"PSC 6217, Box 2610\nAPO AA 53585",kathleen36@gmail.com,1992-07-27,"['62.497506', '2.717198']","317 Lamb Cape Apt. 884\nLake Amy, DC 79074",Patel PLC,Counselling psychologist,34,116095,VISA 16 digit,4854862659569207844,7957,2023-10-31 00:00:00
4,4,6069c2d7-7905-4993-a155-64f6aba143b1,2020-08-09,timothyjackson,Colleen Wheeler,F,"0325 Potter Roads\nLake Lisashire, NM 77502",johnbest@hotmail.com,1989-09-16,"['73.7924695', '-80.314720']","21936 Mary Islands\nMendozafort, TN 37124",Smith-Mejia,Mining engineer,57,107529,JCB 16 digit,213152724828217,72,2023-05-28 00:00:00


## Remove Irrelevant Fields

These two first fields are explicitly mentioned for removal in the text:
	1.	customer_id
	2.	current_location
    3.  Unnamed: 0.  Looks like an error


In [4]:
### Remove Irrelevant Fields 'customer_id', 'current_location'
df.drop(columns=['Unnamed: 0', 'customer_id', 'current_location'], inplace=True)

df.head()


,date_registered,username,name,gender,address,email,birthdate,residence,employer,job,age,salary,credit_card_provider,credit_card_number,credit_card_security_code,credit_card_expire
0,2021-09-29,robertsbryan,Jonathan Snyder,M,"24675 Susan Valley\nNorth Dianabury, MO 02475",marcus58@hotmail.com,1978-03-11,"195 Brandi Junctions\nNew Julieberg, NE 63410","Byrd, Welch and Holt",Chief Technology Officer,49,53979,VISA 19 digit,38985874269846,994,2023-10-27 00:00:00
1,2019-08-17,egarcia,Susan Dominguez,F,"4212 Cheryl Inlet\nPort Davidmouth, NC 54884",alexanderkathy@hotmail.com,1970-11-29,"58272 Brown Isle Apt. 698\nPort Michael, HI 04693",Hurst PLC,Data scientist,43,81510,Discover,6525743622515979,163,2023-07-30 00:00:00
2,2019-11-01,turnermegan,Corey Hebert,M,"07388 Coleman Prairie\nLake Amy, IA 78695",vwood@gmail.com,2009-04-23,"36848 Jones Lane Suite 282\nMarquezbury, ID 26822","Mora, Caldwell and Guerrero",Chief Operating Officer,47,205345,VISA 16 digit,4010729915028682247,634,2023-04-26 00:00:00
3,2021-12-31,richardcampbell,Latasha Griffin,F,"PSC 6217, Box 2610\nAPO AA 53585",kathleen36@gmail.com,1992-07-27,"317 Lamb Cape Apt. 884\nLake Amy, DC 79074",Patel PLC,Counselling psychologist,34,116095,VISA 16 digit,4854862659569207844,7957,2023-10-31 00:00:00
4,2020-08-09,timothyjackson,Colleen Wheeler,F,"0325 Potter Roads\nLake Lisashire, NM 77502",johnbest@hotmail.com,1989-09-16,"21936 Mary Islands\nMendozafort, TN 37124",Smith-Mejia,Mining engineer,57,107529,JCB 16 digit,213152724828217,72,2023-05-28 00:00:00


## Fields to Keep and Actions

Based on the information provided these fields are retained, but they require specific privacy transformations:
| **Field**                   | **Action**                                               |
|-----------------------------|----------------------------------------------------------|
| **date_registered**         | Add random noise to the dates.                           |
| **username**                | Mask the username (e.g., `j****e`).                      |
| **name**                    | Replace with a fake name using a library like `Faker`.   |
| **address**                 | Replace with a fake address.                             |
| **email**                   | Mask the email address (e.g., `j****@domain.com`).       |
| **birthdate**               | Add random noise to the dates.                           |
| **residence**               | Replace with a fake value (e.g., fake city).             |
| **employer**                | Tokenize (e.g., replace with "Employer 1", "Employer 2").|
| **job**                     | Tokenize (e.g., replace with "Job 1", "Job 2").          |
| **age**                     | Categorize into bins (e.g., "18-29", "30-41").           |
| **salary**                  | Categorize into bins (e.g., "Low", "Medium", "High").    |
| **credit_card_provider**    | Tokenize (randomize values while preserving uniqueness). |
| **credit_card_number**      | Mask (e.g., `**** **** **** 1234`).                      |
| **credit_card_security_code** | Fully mask with `****`.                                |
| **credit_card_expire**      | Tokenize (replace with random but unique values).        |

In [5]:
# List the columns in the dataframe
df.columns

Index(['date_registered', 'username', 'name', 'gender', 'address', 'email',
       'birthdate', 'residence', 'employer', 'job', 'age', 'salary',
       'credit_card_provider', 'credit_card_number',
       'credit_card_security_code', 'credit_card_expire'],
      dtype='object')

# Add Noise to Date Columns (date_registered, birthdate)

In [6]:
def add_date_noise(date):
    if pd.isnull(date):
        return None
    noise = random.randint(-365, 365)  # Add up to 1 year of noise
    return (pd.to_datetime(date) + timedelta(days=noise)).date()

df['date_registered'] = df['date_registered'].apply(add_date_noise)
df['birthdate'] = df['birthdate'].apply(add_date_noise)

df.head()

,date_registered,username,name,gender,address,email,birthdate,residence,employer,job,age,salary,credit_card_provider,credit_card_number,credit_card_security_code,credit_card_expire
0,2020-10-25,robertsbryan,Jonathan Snyder,M,"24675 Susan Valley\nNorth Dianabury, MO 02475",marcus58@hotmail.com,1978-02-06,"195 Brandi Junctions\nNew Julieberg, NE 63410","Byrd, Welch and Holt",Chief Technology Officer,49,53979,VISA 19 digit,38985874269846,994,2023-10-27 00:00:00
1,2020-06-21,egarcia,Susan Dominguez,F,"4212 Cheryl Inlet\nPort Davidmouth, NC 54884",alexanderkathy@hotmail.com,1970-07-21,"58272 Brown Isle Apt. 698\nPort Michael, HI 04693",Hurst PLC,Data scientist,43,81510,Discover,6525743622515979,163,2023-07-30 00:00:00
2,2020-06-18,turnermegan,Corey Hebert,M,"07388 Coleman Prairie\nLake Amy, IA 78695",vwood@gmail.com,2008-07-28,"36848 Jones Lane Suite 282\nMarquezbury, ID 26822","Mora, Caldwell and Guerrero",Chief Operating Officer,47,205345,VISA 16 digit,4010729915028682247,634,2023-04-26 00:00:00
3,2021-06-21,richardcampbell,Latasha Griffin,F,"PSC 6217, Box 2610\nAPO AA 53585",kathleen36@gmail.com,1992-01-04,"317 Lamb Cape Apt. 884\nLake Amy, DC 79074",Patel PLC,Counselling psychologist,34,116095,VISA 16 digit,4854862659569207844,7957,2023-10-31 00:00:00
4,2020-07-23,timothyjackson,Colleen Wheeler,F,"0325 Potter Roads\nLake Lisashire, NM 77502",johnbest@hotmail.com,1989-01-13,"21936 Mary Islands\nMendozafort, TN 37124",Smith-Mejia,Mining engineer,57,107529,JCB 16 digit,213152724828217,72,2023-05-28 00:00:00


## Username, name and email

In [7]:

# Initialize the Faker library for generating fake data
fake = Faker()
# ==============================================
# Mask the Username Column
# ==============================================
def mask_username(username):
    return username[0] + "****" + username[-1] if pd.notnull(username) else None

df['username'] = df['username'].apply(mask_username)

# ==============================================
# Replace Name with Fake Name
# ==============================================
df['name'] = [fake.name() for _ in range(len(df))]

# ==============================================
# Mask the Email Column
# ==============================================
def mask_email(email):
    if pd.isnull(email):
        return None
    username, domain = email.split("@")
    return username[0] + "****@" + domain

df['email'] = df['email'].apply(mask_email)

df.head()

,date_registered,username,name,gender,address,email,birthdate,residence,employer,job,age,salary,credit_card_provider,credit_card_number,credit_card_security_code,credit_card_expire
0,2020-10-25,r****n,Kenneth Kramer,M,"24675 Susan Valley\nNorth Dianabury, MO 02475",m****@hotmail.com,1978-02-06,"195 Brandi Junctions\nNew Julieberg, NE 63410","Byrd, Welch and Holt",Chief Technology Officer,49,53979,VISA 19 digit,38985874269846,994,2023-10-27 00:00:00
1,2020-06-21,e****a,William Kidd,F,"4212 Cheryl Inlet\nPort Davidmouth, NC 54884",a****@hotmail.com,1970-07-21,"58272 Brown Isle Apt. 698\nPort Michael, HI 04693",Hurst PLC,Data scientist,43,81510,Discover,6525743622515979,163,2023-07-30 00:00:00
2,2020-06-18,t****n,Ashley Parks,M,"07388 Coleman Prairie\nLake Amy, IA 78695",v****@gmail.com,2008-07-28,"36848 Jones Lane Suite 282\nMarquezbury, ID 26822","Mora, Caldwell and Guerrero",Chief Operating Officer,47,205345,VISA 16 digit,4010729915028682247,634,2023-04-26 00:00:00
3,2021-06-21,r****l,Brett Case,F,"PSC 6217, Box 2610\nAPO AA 53585",k****@gmail.com,1992-01-04,"317 Lamb Cape Apt. 884\nLake Amy, DC 79074",Patel PLC,Counselling psychologist,34,116095,VISA 16 digit,4854862659569207844,7957,2023-10-31 00:00:00
4,2020-07-23,t****n,Bradley Jensen,F,"0325 Potter Roads\nLake Lisashire, NM 77502",j****@hotmail.com,1989-01-13,"21936 Mary Islands\nMendozafort, TN 37124",Smith-Mejia,Mining engineer,57,107529,JCB 16 digit,213152724828217,72,2023-05-28 00:00:00


## Applying the hash function and generalized address to some fields

In [8]:
# ==============================================
# Categorize Salary and Age into Bins
# ==============================================
# Salary bins
salary_bins = [20000, 80000, 130000, 190000, 250000]
salary_labels = ["Low", "Medium", "High", "Very High"]
df['salary_binned'] = pd.cut(df['salary'], bins=salary_bins, labels=salary_labels, include_lowest=True)
df = df.drop(columns=['salary'], errors="ignore")

# Age bins
age_bins = [18, 30, 42, 54, 66]
age_labels = ["18-29", "30-41", "42-53", "54-65"]
df['age_binned'] = pd.cut(df['age'], bins=age_bins, labels=age_labels, include_lowest=True)
df = df.drop(columns=['age'], errors="ignore")

df.head()

,date_registered,username,name,gender,address,email,birthdate,residence,employer,job,credit_card_provider,credit_card_number,credit_card_security_code,credit_card_expire,salary_binned,age_binned
0,2020-10-25,r****n,Kenneth Kramer,M,"24675 Susan Valley\nNorth Dianabury, MO 02475",m****@hotmail.com,1978-02-06,"195 Brandi Junctions\nNew Julieberg, NE 63410","Byrd, Welch and Holt",Chief Technology Officer,VISA 19 digit,38985874269846,994,2023-10-27 00:00:00,Low,42-53
1,2020-06-21,e****a,William Kidd,F,"4212 Cheryl Inlet\nPort Davidmouth, NC 54884",a****@hotmail.com,1970-07-21,"58272 Brown Isle Apt. 698\nPort Michael, HI 04693",Hurst PLC,Data scientist,Discover,6525743622515979,163,2023-07-30 00:00:00,Medium,42-53
2,2020-06-18,t****n,Ashley Parks,M,"07388 Coleman Prairie\nLake Amy, IA 78695",v****@gmail.com,2008-07-28,"36848 Jones Lane Suite 282\nMarquezbury, ID 26822","Mora, Caldwell and Guerrero",Chief Operating Officer,VISA 16 digit,4010729915028682247,634,2023-04-26 00:00:00,Very High,42-53
3,2021-06-21,r****l,Brett Case,F,"PSC 6217, Box 2610\nAPO AA 53585",k****@gmail.com,1992-01-04,"317 Lamb Cape Apt. 884\nLake Amy, DC 79074",Patel PLC,Counselling psychologist,VISA 16 digit,4854862659569207844,7957,2023-10-31 00:00:00,Medium,30-41
4,2020-07-23,t****n,Bradley Jensen,F,"0325 Potter Roads\nLake Lisashire, NM 77502",j****@hotmail.com,1989-01-13,"21936 Mary Islands\nMendozafort, TN 37124",Smith-Mejia,Mining engineer,JCB 16 digit,213152724828217,72,2023-05-28 00:00:00,Medium,54-65


In [9]:
# ==============================================
# Tokenize Credit Card Columns
# ==============================================
def tokenize(value):
    if pd.isnull(value):
        return None
    return hashlib.sha256(str(value).encode()).hexdigest()[:10]  # Tokenize and shorten for simplicity

df['credit_card_provider'] = df['credit_card_provider'].apply(tokenize)
df['credit_card_expire'] = df['credit_card_expire'].apply(tokenize)

# Mask Credit Card Number and Security Code
def mask_credit_card(value):
    if pd.isnull(value):
        return None
    return "**** **** **** " + str(value)[-4:]

df['credit_card_number'] = df['credit_card_number'].apply(mask_credit_card)
df['credit_card_security_code'] = "****"

df.head()

,date_registered,username,name,gender,address,email,birthdate,residence,employer,job,credit_card_provider,credit_card_number,credit_card_security_code,credit_card_expire,salary_binned,age_binned
0,2020-10-25,r****n,Kenneth Kramer,M,"24675 Susan Valley\nNorth Dianabury, MO 02475",m****@hotmail.com,1978-02-06,"195 Brandi Junctions\nNew Julieberg, NE 63410","Byrd, Welch and Holt",Chief Technology Officer,ae568399e6,**** **** **** 9846,****,ab867e0e1a,Low,42-53
1,2020-06-21,e****a,William Kidd,F,"4212 Cheryl Inlet\nPort Davidmouth, NC 54884",a****@hotmail.com,1970-07-21,"58272 Brown Isle Apt. 698\nPort Michael, HI 04693",Hurst PLC,Data scientist,d4a33d5b78,**** **** **** 5979,****,9366ba266b,Medium,42-53
2,2020-06-18,t****n,Ashley Parks,M,"07388 Coleman Prairie\nLake Amy, IA 78695",v****@gmail.com,2008-07-28,"36848 Jones Lane Suite 282\nMarquezbury, ID 26822","Mora, Caldwell and Guerrero",Chief Operating Officer,7e56c08964,**** **** **** 2247,****,fb98978031,Very High,42-53
3,2021-06-21,r****l,Brett Case,F,"PSC 6217, Box 2610\nAPO AA 53585",k****@gmail.com,1992-01-04,"317 Lamb Cape Apt. 884\nLake Amy, DC 79074",Patel PLC,Counselling psychologist,7e56c08964,**** **** **** 7844,****,0ffa10c2f2,Medium,30-41
4,2020-07-23,t****n,Bradley Jensen,F,"0325 Potter Roads\nLake Lisashire, NM 77502",j****@hotmail.com,1989-01-13,"21936 Mary Islands\nMendozafort, TN 37124",Smith-Mejia,Mining engineer,1f49ba014f,**** **** **** 8217,****,0b350e054b,Medium,54-65


In [10]:
# ==============================================
# Tokenize Employer and Job
# ==============================================
unique_employers = df['employer'].dropna().unique()
employer_mapping = {employer: f"Employer {i+1}" for i, employer in enumerate(unique_employers)}
df['employer'] = df['employer'].map(employer_mapping)

unique_jobs = df['job'].dropna().unique()
job_mapping = {job: f"Job {i+1}" for i, job in enumerate(unique_jobs)}
df['job'] = df['job'].map(job_mapping)

df.head()

,date_registered,username,name,gender,address,email,birthdate,residence,employer,job,credit_card_provider,credit_card_number,credit_card_security_code,credit_card_expire,salary_binned,age_binned
0,2020-10-25,r****n,Kenneth Kramer,M,"24675 Susan Valley\nNorth Dianabury, MO 02475",m****@hotmail.com,1978-02-06,"195 Brandi Junctions\nNew Julieberg, NE 63410",Employer 1,Job 1,ae568399e6,**** **** **** 9846,****,ab867e0e1a,Low,42-53
1,2020-06-21,e****a,William Kidd,F,"4212 Cheryl Inlet\nPort Davidmouth, NC 54884",a****@hotmail.com,1970-07-21,"58272 Brown Isle Apt. 698\nPort Michael, HI 04693",Employer 2,Job 2,d4a33d5b78,**** **** **** 5979,****,9366ba266b,Medium,42-53
2,2020-06-18,t****n,Ashley Parks,M,"07388 Coleman Prairie\nLake Amy, IA 78695",v****@gmail.com,2008-07-28,"36848 Jones Lane Suite 282\nMarquezbury, ID 26822",Employer 3,Job 3,7e56c08964,**** **** **** 2247,****,fb98978031,Very High,42-53
3,2021-06-21,r****l,Brett Case,F,"PSC 6217, Box 2610\nAPO AA 53585",k****@gmail.com,1992-01-04,"317 Lamb Cape Apt. 884\nLake Amy, DC 79074",Employer 4,Job 4,7e56c08964,**** **** **** 7844,****,0ffa10c2f2,Medium,30-41
4,2020-07-23,t****n,Bradley Jensen,F,"0325 Potter Roads\nLake Lisashire, NM 77502",j****@hotmail.com,1989-01-13,"21936 Mary Islands\nMendozafort, TN 37124",Employer 5,Job 5,1f49ba014f,**** **** **** 8217,****,0b350e054b,Medium,54-65


In [11]:
# ==============================================
# Replace Residence and Address with Fake Values
# ==============================================
df['residence'] = [fake.city() for _ in range(len(df))]
df['address'] = [fake.address().replace("\n", ", ") for _ in range(len(df))]

df.head()

,date_registered,username,name,gender,address,email,birthdate,residence,employer,job,credit_card_provider,credit_card_number,credit_card_security_code,credit_card_expire,salary_binned,age_binned
0,2020-10-25,r****n,Kenneth Kramer,M,"7017 Ware Island, Martinstad, AS 58136",m****@hotmail.com,1978-02-06,Lake Kelly,Employer 1,Job 1,ae568399e6,**** **** **** 9846,****,ab867e0e1a,Low,42-53
1,2020-06-21,e****a,William Kidd,F,"2269 Monica Circle, Lake Amanda, MT 71346",a****@hotmail.com,1970-07-21,Riddleberg,Employer 2,Job 2,d4a33d5b78,**** **** **** 5979,****,9366ba266b,Medium,42-53
2,2020-06-18,t****n,Ashley Parks,M,"208 Abigail Club, South Tonyview, IL 90526",v****@gmail.com,2008-07-28,Greenchester,Employer 3,Job 3,7e56c08964,**** **** **** 2247,****,fb98978031,Very High,42-53
3,2021-06-21,r****l,Brett Case,F,"8281 Adams Cliffs, North John, HI 08946",k****@gmail.com,1992-01-04,Lake Meganton,Employer 4,Job 4,7e56c08964,**** **** **** 7844,****,0ffa10c2f2,Medium,30-41
4,2020-07-23,t****n,Bradley Jensen,F,"0718 Amanda Hollow, Port Jameston, PA 77188",j****@hotmail.com,1989-01-13,Batesfurt,Employer 5,Job 5,1f49ba014f,**** **** **** 8217,****,0b350e054b,Medium,54-65


## Exporting the data to send to InsightSpark

In [13]:
# Save the anonymized data to a new file
df.to_csv("result/anonymized_mobile_customers.csv", index=False)